# **Dependências do Projeto**

In [1]:
%pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch torchvision torchaudio
#Instalando o pytorch

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
#Caso não consiga pydirectinput
%pip install pyautogui

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Biblioteca de aprendizado por reforço
%pip install stable-baselines3[extra] protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install mss pydirectinput pytesseract

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install optuna

# **Imports**

In [21]:
#Para captura de tela
from mss import mss
#import pyautogui #para comandos de teclado
import cv2 
#Tratamentos dos frames
import numpy as np 
import pyautogui
#Framework intermediário paara trabalhar com as imagens
#import pytesseract #para extrair o game over da imagem
from matplotlib import pyplot as plt #Visualizando resultados
import time
from PIL import Image
#Coisas para construir o ambinete
from gym import Env
from gym.spaces import Box, Discrete, Dict
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from io import BytesIO
from PIL import Image
from selenium.common.exceptions import WebDriverException
import base64
from io import BytesIO


In [2]:
%load_ext tensorboard

# **Construindo o Ambiente**

In [3]:
from collections import deque

In [4]:
#Classe para definir o ambiente
class WebGame(Env):
    #ESSAS FUNÇÃO SÃO NECESSÁRIAS POR CONTA DO GYM
    #Função responsavel pela inicialização do ambiente;
    #Portanto onde ficam todas as variaveis do ambiente, configuração das ações e da observação 
    def __init__(self):
        super().__init__()

        #Configuração do tempo
        self.print_time = .005
        self.wait_time = .015
        
        #Configuração do espaço
        self.observation_space = Dict({
            "print":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
            "score":Box(low=0, high=10000, shape=(1,), dtype=np.uint8),
            })
        self.action_space = Discrete(2)

        #Variavei para a captura de tela
        self.screen_cap = mss() #Lib de captura de tela
        self.game_location = {"top": 300, "left": 0, "width": 650, "height": 500} #Área do monitor com 
        self.game_done = {"top": 40, "left": 0, "width": 800, "height": 640}

        self.chromedriver_path = "./chromedriver.exe"
        _chrome_options = webdriver.ChromeOptions()
        _chrome_options.add_argument("--mute-audio")
        _chrome_options.add_argument("--disable-gpu")
        self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)

        self.last_image = np.zeros((500,650,3))
        
        #Dicionário de ações
        self.action_dict = {
            0:Keys.ARROW_UP,
            1:'nothing'
        }

    #Função responsavel por passar as ações pro jogo, para fazer algo pro jogo
    def step(self, action):
        # Se ação escolhida for um dos botões, realizar a ação.
        if action != 1:
            self._driver.find_elements(By.TAG_NAME, "body")[0].send_keys(self.action_dict[int(action)])
        
        #Checa a próxima ação
        observation = self.get_observation()
        #Todas as ações tem que verificar se a ação acabou
        done, done_cap = self.get_done() 
        
        #Pegando o score do jogo
        score = self.get_score()
        #Ganhamos 1 ponto por cada frame que estamos vivos.
        reward = 1
        #É um dicionário de informações que retornam através do que a gente precisa
        info = {
            'score':score
        }

        time.sleep(self.wait_time)
        
        return observation, reward, done, info
        

    #Função para a visualização do jogo:
    def render(self, mode: str='human'):
        img = cv2.cvtColor(self._get_image(), cv2.COLOR_BGR2RGB)
        if mode == 'rgb_array':
            return img
        elif mode == 'human':
            from gym.envs.classic_control import rendering
            if self.viewer is None:
                self.viewer = rendering.SimpleImageViewer()
            self.viewer.imshow(img)
            return self.viewer.isopen

        # cv2.imshow('Game', np.array(self.cap.grab(self.game_location))[:,:,:3])
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     self.close()

    #Função responsavel por recomeçar o jogo
    def reset(self):
        #Try catch precisa existir pq toda a vez q o selenium detecta que ele viu o chrome dino,
        #ele dispara um erro que esta sem internet.
        try:
            self._driver.get('chrome://dino')
            
        except WebDriverException:
            pass
        WebDriverWait(self._driver, 10).until(
            EC.presence_of_element_located((
                By.CLASS_NAME, 
                "runner-canvas"
            ))
        )
        self._driver.implicitly_wait(0.5)
        self._driver.find_elements(By.TAG_NAME, "body")[0].send_keys(Keys.ARROW_UP)

        return self.get_observation()

    #FUNÇÕES CUSTOM
    #Fecha a parte de visualização
    def close(self):
        cv2.destroyAllWindows()
    
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array)
        return int(score)
    
    def get_score_shape(self):
        shape = np.zeros(1)
        shape[0] = int(self.get_score())
        return shape
    
    def get_img(self):
        LEADING_TEXT = "data:image/png;base64,"
        canvas = self._driver.execute_script("return document.querySelector('canvas.runner-canvas').toDataURL()")
        img = canvas[len(LEADING_TEXT):]
        img_data = np.array(Image.open(BytesIO(base64.b64decode(img))))
        return img_data 
        # img = self._driver.execute_script()
    
    #Função responsavel por pegar a imagem do jogo e tratar da forma necessária.
    def get_print(self): 

        #img = numpy.array(self.screen_cap.grab(game_location))[:,:,3].astype(np.uint8)
        raw = np.array(self.get_img())[:,:,:3].astype(np.uint8)
        img = raw[:200,:400]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (200,83))
        channel = np.reshape(resized, (1, 83,200))
        self.last_image = raw
        
        return channel
    
    #Função responsavel por pegar a imagem do jogo e tratar da forma necessária.
    def get_observation(self): 
        return {"print":self.get_print(), "score":self.get_score_shape() }


    #Função para pegar o texto de fim de jogo:
    def get_done(self):
        return self._driver.execute_script("return Runner.instance_.crashed"), self.last_image

# Treinamento

In [5]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback, EventCallback
# Verificando se o ambiente é válido para fazer as coisa
from stable_baselines3.common import env_checker

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv, VecNormalize

In [6]:
from stable_baselines3 import DQN, PPO, DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack,SubprocVecEnv
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.logger import Figure, configure

In [10]:
def get_env():
    env_lambda = lambda: Monitor(WebGame())
    env = DummyVecEnv([env_lambda])
    #env = SubprocVecEnv([env_lambda for i in range(4)])
    #env = VecFrameStack(env, 4, channels_order='first')
    #env = VecNormalize(env, norm_obs=True, norm_reward=True,
    #               clip_obs=10.)
    env = Monitor(WebGame())
    return env

In [11]:
#env_lambda = lambda: get_env()
#env2 = SubprocVecEnv([env_lambda for i in range(2)])
env2 = get_env()

C:\Users\Dan\AppData\Local\Temp\ipykernel_15156\3679928894.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)


In [12]:
env2.reset()

{'print': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8),
 'score': array([0.])}

In [13]:
#Criandos as pastas para que funcione.
CHECKPOINT_DIR = './checkpoints/'
LOG_DIR = './logs/'

In [14]:
#Salvando o modelo em uma pasta chamado de treino
import os
import warnings
from abc import ABC, abstractmethod
from typing import Any, Callable, Dict, List, Optional, Union

import gym
import numpy as np

from stable_baselines3.common.logger import Logger

from stable_baselines3.common import base_class  # pytype: disable=pyi-error
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecEnv, sync_envs_normalization

class EvalCallbackButBetter(EventCallback):
    def __init__(
        self,
        eval_env: Union[gym.Env, VecEnv],
        callback_on_new_best: Optional[BaseCallback] = None,
        callback_after_eval: Optional[BaseCallback] = None,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        log_path: Optional[str] = None,
        best_model_save_path: Optional[str] = None,
        deterministic: bool = True,
        render: bool = False,
        verbose: int = 1,
        warn: bool = True,
    ):
        super().__init__(callback_after_eval, verbose=verbose)

        self.callback_on_new_best = callback_on_new_best
        if self.callback_on_new_best is not None:
            # Give access to the parent
            self.callback_on_new_best.parent = self

        self.n_eval_episodes = n_eval_episodes
        self.eval_freq = eval_freq
        self.best_mean_reward = -np.inf
        self.last_mean_reward = -np.inf
        self.deterministic = deterministic
        self.render = render
        self.warn = warn

        # Convert to VecEnv for consistency
        if not isinstance(eval_env, VecEnv):
            eval_env = DummyVecEnv([lambda: eval_env])

        self.eval_env = eval_env
        self.best_model_save_path = best_model_save_path
        # Logs will be written in ``evaluations.npz``
        if log_path is not None:
            log_path = os.path.join(log_path, "evaluations")
        self.log_path = log_path
        self.evaluations_results = []
        self.evaluations_timesteps = []
        self.evaluations_length = []
        # For computing success rate
        self._is_success_buffer = []
        self.evaluations_successes = []

    def _init_callback(self) -> None:
        # Does not work in some corner cases, where the wrapper is not the same
        if not isinstance(self.training_env, type(self.eval_env)):
            warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")

        # Create folders if needed
        if self.best_model_save_path is not None:
            os.makedirs(self.best_model_save_path, exist_ok=True)
        if self.log_path is not None:
            os.makedirs(os.path.dirname(self.log_path), exist_ok=True)

        # Init callback called on new best model
        if self.callback_on_new_best is not None:
            self.callback_on_new_best.init_callback(self.model)

    def _log_success_callback(self, locals_: Dict[str, Any], globals_: Dict[str, Any]) -> None:
        """
        Callback passed to the  ``evaluate_policy`` function
        in order to log the success rate (when applicable),
        for instance when using HER.
        :param locals_:
        :param globals_:
        """
        info = locals_["info"]

        if locals_["done"]:
            maybe_is_success = info.get("is_success")
            if maybe_is_success is not None:
                self._is_success_buffer.append(maybe_is_success)

    def _on_step(self) -> bool:
        continue_training = True

        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Sync training and eval env if there is VecNormalize
            if self.model.get_vec_normalize_env() is not None:
                try:
                    sync_envs_normalization(self.training_env, self.eval_env)
                except AttributeError as e:
                    raise AssertionError(
                        "Training and eval env are not wrapped the same way, "
                        "see https://stable-baselines3.readthedocs.io/en/master/guide/callbacks.html#evalcallback "
                        "and warning above."
                    ) from e

            # Reset success rate buffer
            self._is_success_buffer = []

            episode_rewards, episode_lengths = evaluate_policy(
                self.model,
                self.eval_env,
                n_eval_episodes=self.n_eval_episodes,
                render=self.render,
                deterministic=self.deterministic,
                return_episode_rewards=True,
                warn=self.warn,
                callback=self._log_success_callback,
            )

            if self.log_path is not None:
                self.evaluations_timesteps.append(self.num_timesteps)
                self.evaluations_results.append(episode_rewards)
                self.evaluations_length.append(episode_lengths)

                kwargs = {}
                # Save success log if present
                if len(self._is_success_buffer) > 0:
                    self.evaluations_successes.append(self._is_success_buffer)
                    kwargs = dict(successes=self.evaluations_successes)

                np.savez(
                    self.log_path,
                    timesteps=self.evaluations_timesteps,
                    results=self.evaluations_results,
                    ep_lengths=self.evaluations_length,
                    **kwargs,
                )

            mean_reward, std_reward = np.mean(episode_rewards), np.std(episode_rewards)
            mean_ep_length, std_ep_length = np.mean(episode_lengths), np.std(episode_lengths)
            self.last_mean_reward = mean_reward

            if self.verbose >= 1:
                print(f"Eval num_timesteps={self.num_timesteps}, " f"episode_reward={mean_reward:.2f} +/- {std_reward:.2f}")
                print(f"Episode length: {mean_ep_length:.2f} +/- {std_ep_length:.2f}")
            # Add to current Logger
            self.logger.record("eval/mean_reward", float(mean_reward))
            self.logger.record("eval/mean_ep_length", mean_ep_length)

            if len(self._is_success_buffer) > 0:
                success_rate = np.mean(self._is_success_buffer)
                if self.verbose >= 1:
                    print(f"Success rate: {100 * success_rate:.2f}%")
                self.logger.record("eval/success_rate", success_rate)

            # Dump log so the evaluation results are printed with the correct timestep
            self.logger.record("time/total_timesteps", self.num_timesteps, exclude="tensorboard")
            self.logger.dump(self.num_timesteps)

            if mean_reward > self.best_mean_reward:
                if self.verbose >= 1:
                    print("New best mean reward!")
                if self.best_model_save_path is not None:
                    self.model.save(os.path.join(self.best_model_save_path, "best_model"))
                    self.model.save_replay_buffer(os.path.join(self.best_model_save_path, "best_model_buffer"))
                    self.model.policy.save(os.path.join(self.best_model_save_path, "best_policy"))
                    #self.model.get_env().env_method("save", [os.path.join(self.best_model_save_path, "best_model_env")])      
                    
                self.best_mean_reward = mean_reward
                # Trigger callback on new best model, if needed
                if self.callback_on_new_best is not None:
                    continue_training = self.callback_on_new_best.on_step()

            # Trigger callback after every evaluation, if needed
            if self.callback is not None:
                continue_training = continue_training and self._on_event()

        return continue_training

    def update_child_locals(self, locals_: Dict[str, Any]) -> None:
        """
        Update the references to the local variables.
        :param locals_: the local variables during rollout collection
        """
        if self.callback:
            self.callback.update_locals(locals_)

In [15]:
callback = EvalCallbackButBetter(eval_env=env2, verbose=2, best_model_save_path=CHECKPOINT_DIR,
                             log_path=LOG_DIR, eval_freq=2000,
                             deterministic=True, render=False)

In [16]:
def get_model():
    #model = PPO('MultiInputPolicy', env2, tensorboard_log=LOG_DIR, learning_rate=0.005, verbose=2, policy_kwargs=dict(normalize_images=False))
    model = DQN('MultiInputPolicy', env2, tensorboard_log=LOG_DIR, verbose=2, buffer_size=12000, learning_rate=0.0006, 
        learning_starts=500, exploration_fraction=0.55, exploration_initial_eps=0.89, exploration_final_eps=0.07,
                policy_kwargs=dict(normalize_images=False))
    model.set_random_seed(42)
    return model

In [17]:
model = get_model()

Using cpu device
Wrapping the env in a DummyVecEnv.


In [18]:
#Começa o treinamento
model._last_obs = None
model.learn(total_timesteps=45000, callback=callback, reset_num_timesteps=False)

Logging to ./logs/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 182      |
|    ep_rew_mean      | 182      |
|    exploration_rate | 0.866    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 22       |
|    time_elapsed     | 31       |
|    total_timesteps  | 726      |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00175  |
|    n_updates        | 56       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 164      |
|    ep_rew_mean      | 164      |
|    exploration_rate | 0.847    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 20       |
|    time_elapsed     | 62       |
|    total_timesteps  | 1312     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00118

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 116      |
|    ep_rew_mean      | 116      |
|    exploration_rate | 0.665    |
| time/               |          |
|    episodes         | 56       |
|    fps              | 14       |
|    time_elapsed     | 463      |
|    total_timesteps  | 6793     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.000785 |
|    n_updates        | 1573     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 115      |
|    ep_rew_mean      | 115      |
|    exploration_rate | 0.651    |
| time/               |          |
|    episodes         | 60       |
|    fps              | 14       |
|    time_elapsed     | 485      |
|    total_timesteps  | 7209     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.000431 |
|    n_updates      

E:\anaconda3\lib\site-packages\stable_baselines3\common\save_util.py:272: UserWarning: Path 'checkpoints\best_model_buffer.pkl' exists, will overwrite it.
  warnings.warn(f"Path '{path}' exists, will overwrite it.")


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 112      |
|    ep_rew_mean      | 112      |
|    exploration_rate | 0.549    |
| time/               |          |
|    episodes         | 88       |
|    fps              | 14       |
|    time_elapsed     | 721      |
|    total_timesteps  | 10284    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00221  |
|    n_updates        | 2445     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 113      |
|    ep_rew_mean      | 113      |
|    exploration_rate | 0.531    |
| time/               |          |
|    episodes         | 92       |
|    fps              | 14       |
|    time_elapsed     | 750      |
|    total_timesteps  | 10822    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00364  |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 114      |
|    ep_rew_mean      | 114      |
|    exploration_rate | 0.338    |
| time/               |          |
|    episodes         | 140      |
|    fps              | 14       |
|    time_elapsed     | 1150     |
|    total_timesteps  | 16656    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0013   |
|    n_updates        | 4038     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 115      |
|    ep_rew_mean      | 115      |
|    exploration_rate | 0.32     |
| time/               |          |
|    episodes         | 144      |
|    fps              | 14       |
|    time_elapsed     | 1179     |
|    total_timesteps  | 17197    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00169  |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 118      |
|    ep_rew_mean      | 118      |
|    exploration_rate | 0.127    |
| time/               |          |
|    episodes         | 192      |
|    fps              | 14       |
|    time_elapsed     | 1577     |
|    total_timesteps  | 23021    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00499  |
|    n_updates        | 5630     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 118      |
|    ep_rew_mean      | 118      |
|    exploration_rate | 0.114    |
| time/               |          |
|    episodes         | 196      |
|    fps              | 14       |
|    time_elapsed     | 1600     |
|    total_timesteps  | 23430    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.000844 |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 116      |
|    ep_rew_mean      | 116      |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 244      |
|    fps              | 14       |
|    time_elapsed     | 1997     |
|    total_timesteps  | 29078    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00116  |
|    n_updates        | 7144     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 116      |
|    ep_rew_mean      | 116      |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 248      |
|    fps              | 14       |
|    time_elapsed     | 2026     |
|    total_timesteps  | 29606    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0387   |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 119      |
|    ep_rew_mean      | 119      |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 296      |
|    fps              | 14       |
|    time_elapsed     | 2466     |
|    total_timesteps  | 35698    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00604  |
|    n_updates        | 8799     |
----------------------------------
Eval num_timesteps=36000, episode_reward=161.00 +/- 33.62
Episode length: 161.00 +/- 33.62
----------------------------------
| eval/               |          |
|    mean_ep_length   | 161      |
|    mean_reward      | 161      |
| rollout/            |          |
|    exploration_rate | 0.07     |
| time/               |          |
|    total_timesteps  | 36000    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00927  |

Eval num_timesteps=42000, episode_reward=138.80 +/- 26.72
Episode length: 138.80 +/- 26.72
----------------------------------
| eval/               |          |
|    mean_ep_length   | 139      |
|    mean_reward      | 139      |
| rollout/            |          |
|    exploration_rate | 0.07     |
| time/               |          |
|    total_timesteps  | 42000    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0034   |
|    n_updates        | 10374    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 120      |
|    ep_rew_mean      | 120      |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 348      |
|    fps              | 14       |
|    time_elapsed     | 2960     |
|    total_timesteps  | 42113    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0183   |

## Otimização de hiper parametros

In [11]:
#Biblioteca para isso
import optuna

In [12]:
#Criandos as pastas para que funcione.
CHECKPOINT_DIR = './checkpoints/'
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [13]:
#Função para tentar retornar os hiper parametros de teste e definir a função objetivo
#ALTERAR:
#Essa função deve ser modificada com os respectivos Hiper-Parametros desejados para teste
def otimizar_modelo(trial):
    return {
        'learning_starts': trial.suggest_int('learning_starts', 500, 1000),
        'exploration_fraction': trial.suggest_float('exploration_fraction', 0.4, 0.6),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-4),
        'exploration_initial_eps': trial.suggest_float('exploration_initial_eps', 0.8, 0.95),
        'exploration_final_eps': trial.suggest_float('exploration_final_eps', 0.05, 0.1),
    }

In [14]:
#Função para testar hiper parametros
def optimize_agent(trial):
    try:
        model_params = otimizar_modelo(trial)
        env2 = get_env()
        #ALTERAR:
        #Alterar o modelo para oq precisa. A única coisa que deve se manter é o '**model_params' pois isso é responsavel para passar para os modelos os hiper parametros necessários para otimização.
        model = DQN('MultiInputPolicy', env2, tensorboard_log=LOG_DIR, verbose=2, buffer_size=12000, **model_params,
                policy_kwargs=dict(normalize_images=False))
        model.set_random_seed(42)
        
        callback = EvalCallbackButBetter(eval_env=env2, verbose=2, best_model_save_path=CHECKPOINT_DIR,
                             log_path=LOG_DIR, eval_freq=2000,
                             deterministic=True, render=False)
        
        model.learn(total_timesteps=5000, callback=callback)
        
        mean_reward,_ = evaluate_policy(model, env2, n_eval_episodes=5)
        env2.close()

        return mean_reward

    except Exception as e:
        print(e)
        return -1000

In [15]:
study = optuna.create_study(direction='maximize')
#Parametros:
#Função a ser otimizada,
#Número de tentativas de teste,
#N_jobs: número de processos em paralelos,
#gc_after_trials: garbage collector para que meu pc n exploda sem memória.
study.optimize(optimize_agent, n_trials=10, n_jobs=1, gc_after_trial=True)

[I 2023-03-16 13:54:29,572] A new study created in memory with name: no-name-21c35718-f2c8-43bd-abda-4972360c7a05


Using cpu device
Logging to ./logs/DQN_34
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 110      |
|    ep_rew_mean      | 110      |
|    exploration_rate | 0.667    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 63       |
|    time_elapsed     | 8        |
|    total_timesteps  | 560      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 104      |
|    ep_rew_mean      | 104      |
|    exploration_rate | 0.534    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 57       |
|    time_elapsed     | 15       |
|    total_timesteps  | 900      |
| train/              |          |
|    learning_rate    | 1.51e-05 |
|    loss             | 0.123    |
|    n_updates        | 13       |
----------------------------------
----------------------------------
| rollout/   

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 72.4     |
|    ep_rew_mean      | 72.4     |
|    exploration_rate | 0.0946   |
| time/               |          |
|    episodes         | 68       |
|    fps              | 43       |
|    time_elapsed     | 114      |
|    total_timesteps  | 4952     |
| train/              |          |
|    learning_rate    | 1.51e-05 |
|    loss             | 0.0209   |
|    n_updates        | 266      |
----------------------------------


[I 2023-03-16 13:56:46,503] Trial 0 finished with value: 94.4 and parameters: {'n_steps': 701, 'gamma': 0.4043753087646491, 'learning_rate': 1.509016214991424e-05, 'exploration_initial_eps': 0.886004688920427, 'exploration_final_eps': 0.09458918516342053}. Best is trial 0 with value: 94.4.


Using cpu device
Logging to ./logs/DQN_35
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 104      |
|    ep_rew_mean      | 104      |
|    exploration_rate | 0.773    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 59       |
|    time_elapsed     | 7        |
|    total_timesteps  | 428      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 95.8     |
|    ep_rew_mean      | 95.8     |
|    exploration_rate | 0.606    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 53       |
|    time_elapsed     | 16       |
|    total_timesteps  | 872      |
| train/              |          |
|    learning_rate    | 6.09e-05 |
|    loss             | 0.0632   |
|    n_updates        | 19       |
----------------------------------
----------------------------------
| rollout/   

...

Using cpu device
Logging to ./logs/DQN_42
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 140      |
|    ep_rew_mean      | 140      |
|    exploration_rate | 0.727    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 63       |
|    time_elapsed     | 10       |
|    total_timesteps  | 688      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 123      |
|    ep_rew_mean      | 123      |
|    exploration_rate | 0.604    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 61       |
|    time_elapsed     | 17       |
|    total_timesteps  | 1092     |
| train/              |          |
|    learning_rate    | 9.99e-05 |
|    loss             | 0.196    |
|    n_updates        | 6        |
----------------------------------
----------------------------------
| rollout/   

[I 2023-03-16 14:14:51,069] Trial 8 finished with value: 95.2 and parameters: {'n_steps': 993, 'gamma': 0.571060241080772, 'learning_rate': 9.989457366073071e-05, 'exploration_initial_eps': 0.937545622455347, 'exploration_final_eps': 0.06424600649047998}. Best is trial 7 with value: 124.8.


Using cpu device
Logging to ./logs/DQN_43
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 106      |
|    ep_rew_mean      | 106      |
|    exploration_rate | 0.681    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 58       |
|    time_elapsed     | 7        |
|    total_timesteps  | 444      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 96.1     |
|    ep_rew_mean      | 96.1     |
|    exploration_rate | 0.571    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 54       |
|    time_elapsed     | 14       |
|    total_timesteps  | 804      |
| train/              |          |
|    learning_rate    | 3.11e-05 |
|    loss             | 0.113    |
|    n_updates        | 15       |
----------------------------------
----------------------------------
| rollout/   

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 68.1     |
|    ep_rew_mean      | 68.1     |
|    exploration_rate | 0.0606   |
| time/               |          |
|    episodes         | 68       |
|    fps              | 43       |
|    time_elapsed     | 109      |
|    total_timesteps  | 4708     |
| train/              |          |
|    learning_rate    | 3.11e-05 |
|    loss             | 0.00752  |
|    n_updates        | 259      |
----------------------------------


[I 2023-03-16 14:17:07,080] Trial 9 finished with value: 98.6 and parameters: {'n_steps': 570, 'gamma': 0.4942287335726097, 'learning_rate': 3.105770103514228e-05, 'exploration_initial_eps': 0.8171451372237063, 'exploration_final_eps': 0.06064541984243099}. Best is trial 7 with value: 124.8.


# Teste

In [22]:
from stable_baselines3.dqn.policies import MultiInputPolicy

env2 = get_env()
model = get_model()
model.load('checkpoints/best_model') 
model.load_replay_buffer('checkpoints/best_model_buffer') 
saved_policy = MultiInputPolicy.load("checkpoints/best_policy")

# Evaluate the loaded policy
mean_reward, std_reward = evaluate_policy(saved_policy, env2, n_eval_episodes=10, deterministic=True)
print('mean_reward: {}'.format(mean_reward))
print('std_reward: {}'.format(std_reward))

    

C:\Users\Dan\AppData\Local\Temp\ipykernel_15156\3679928894.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)


Using cpu device
Wrapping the env in a DummyVecEnv.


E:\anaconda3\lib\site-packages\stable_baselines3\common\save_util.py:263: UserWarning: Path 'checkpoints\best_model_buffer' not found. Attempting checkpoints\best_model_buffer.pkl.
  warnings.warn(f"Path '{path}' not found. Attempting {newpath}.")


mean_reward: 267.1
std_reward: 175.97639046190258


In [19]:
%tensorboard --logdir ./logs/DQN_0/ --host localhost